In [2]:
import numpy as np 
import pandas as pd 

In [ ]:
movies= pd.read_csv("movies.csv")
movies


In [ ]:
ratings= pd.read_csv("ratings.csv")

ratings

In [ ]:
df=pd.merge(movies,ratings)
df.drop(["genres","timestamp"], axis = 1,inplace = True)
df
#using only the movies and ratings dataset to feed into the model and 
#filtering out the unwanted columns as they wont be affecting the recommendation

In [ ]:
finaldf = df.pivot_table(index='movieId',columns='userId',values='rating')
finaldf.fillna(0,inplace=True)
finaldf
#creating a table with required values and filling Null with 0

In [7]:
no_user_voted = ratings.groupby('movieId')['rating'].agg('count')
no_movies_voted = ratings.groupby('userId')['rating'].agg('count')

In [ ]:
finaldf=finaldf.loc[:,no_movies_voted[no_movies_voted > 50].index]
finaldf

In [9]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
#knn is the most suitable algorithm as the values aren't necessarily completely and linearly dependent on one another
#knn assumes the similarity between the new case/data and available cases and put the new case into the category that is most similar to the available categories.

In [10]:
csr_data = csr_matrix(finaldf.values)
finaldf.reset_index(inplace=True)

In [26]:
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=25, n_jobs=-1)
knn.fit(csr_data)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=25)

In [30]:
def get_movie_recommendation(movie_name):
    n_movies_to_reccomend = 5
    movie_list = movies[movies['title'].str.contains(movie_name)]  
    if len(movie_list):        
        movie_idx= movie_list.iloc[0]['movieId']
        movie_idx = finaldf[finaldf['movieId'] == movie_idx].index[0]
        distances , indices = knn.kneighbors(csr_data[movie_idx],n_neighbors=n_movies_to_reccomend+1)    
        rec_movie_indices = sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x: x[1])[:0:-1]
        recommend_frame = []
        for val in rec_movie_indices:
            movie_idx = finaldf.iloc[val[0]]['movieId']
            idx = movies[movies['movieId'] == movie_idx].index
            recommend_frame.append({'Title':movies.iloc[idx]['title'].values[0],'Distance':val[1]})
        df = pd.DataFrame(recommend_frame,index=range(1,n_movies_to_reccomend+1))
        return df
    else:
        return "Invalid movie input"

In [32]:
get_movie_recommendation('Dark Knight')

,Title,Distance
1,"Lord of the Rings: The Return of the King, The...",0.347528
2,"Dark Knight Rises, The (2012)",0.308688
3,Batman Begins (2005),0.306755
4,Iron Man (2008),0.285835
5,Inception (2010),0.213876
